In [1]:
import zipfile

zip_ref = zipfile.ZipFile("all.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

zip_ref = zipfile.ZipFile("train.zip", 'r')
zip_ref.extractall(".")
zip_ref = zipfile.ZipFile("test.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

import os
import shutil

train_filenames = os.listdir('train')
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)

def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)
    
rmrf_mkdir('trainRun')
os.mkdir('trainRun/cat')
os.mkdir('trainRun/dog')

rmrf_mkdir('testRun')
os.symlink('../test/', 'testRun/test')

for filename in train_cat:
    os.symlink('../../train/'+filename, 'trainRun/cat/'+filename)

for filename in train_dog:
    os.symlink('../../train/'+filename, 'trainRun/dog/'+filename)
    

In [2]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image

import h5py

base_model = InceptionResNetV2(weights='imagenet', include_top=False)
model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


219062272/219055592 [==============================] - 3s 0us/step


In [3]:
img_width, img_height = 299, 299
train_datagen = image.ImageDataGenerator(shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = image.ImageDataGenerator()
train_generator = train_datagen.flow_from_directory("trainRun", target_size=(img_height, img_width), shuffle=False, batch_size=16)
test_generator = test_datagen.flow_from_directory("testRun", target_size=(img_height, img_width), shuffle=False, batch_size=16, class_mode=None)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [4]:
train_h5py = model.predict_generator(train_generator)
test_h5py = model.predict_generator(test_generator)

In [5]:
with h5py.File("init_weights_InceptionRestNetV2.h5") as h:
        h.create_dataset("train", data=train_h5py)
        h.create_dataset("test", data=test_h5py)
        h.create_dataset("label", data=train_generator.classes)

In [6]:
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

with h5py.File("init_weights_InceptionRestNetV2.h5", 'r') as h:
    X_train.append(np.array(h['train']))
    X_test.append(np.array(h['test']))
    y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [18]:
from keras.layers import Input, Dropout, Dense
np.random.seed(2017)

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model_run = Model(input_tensor, x)

model_run.compile(optimizer='adadelta',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

In [19]:
model_run.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 3s 139us/step - loss: 7.9308 - acc: 0.4979 - val_loss: 7.9688 - val_acc: 0.5056
Epoch 2/8
20000/20000 [==============================] - 1s 33us/step - loss: 8.0072 - acc: 0.4930 - val_loss: 8.0605 - val_acc: 0.4944
Epoch 3/8
20000/20000 [==============================] - 1s 34us/step - loss: 7.8586 - acc: 0.5020 - val_loss: 8.0605 - val_acc: 0.4944
Epoch 4/8
20000/20000 [==============================] - 1s 33us/step - loss: 7.8004 - acc: 0.5064 - val_loss: 8.0605 - val_acc: 0.4944
Epoch 5/8
20000/20000 [==============================] - 1s 34us/step - loss: 7.9662 - acc: 0.4969 - val_loss: 7.9688 - val_acc: 0.5056
Epoch 6/8
20000/20000 [==============================] - 1s 33us/step - loss: 7.8433 - acc: 0.5048 - val_loss: 7.9688 - val_acc: 0.5056
Epoch 7/8
20000/20000 [==============================] - 1s 33us/step - loss: 7.8009 - acc: 0.5101 - val_loss: 7.9688 - val_acc: 0.505

In [11]:
y_pred = model_run.predict(X_test, verbose=1)
# y_pred = y_pred.clip(min=0.003, max=0.997)

import pandas as pd

df = pd.read_csv("sample_submission.csv")

test_run_datagen = image.ImageDataGenerator()
test_run_generator = test_run_datagen.flow_from_directory("testRun", 
                                                          (img_height, img_width), 
                                                         shuffle=False, 
                                                         batch_size=16, 
                                                         class_mode=None)

for i, fname in enumerate(test_run_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred_IceRNV2.csv', index=None)
df.head(10)

12500/12500 [==============================] - 1s 44us/step
Found 12500 images belonging to 1 classes.


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,id,label
0,1,1.0
1,2,1.0
2,3,1.0
3,4,1.0
4,5,1.0
5,6,1.0
6,7,1.0
7,8,1.0
8,9,1.0
9,10,1.0
